In [357]:
import pandas as pd
import requests
import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt
from itertools import islice
from os.path import exists

pd.set_option('display.max_colwidth', None)

In [426]:
class MyPortfolio:
    def __init__(self, 
                 date_start, 
                 date_end, 
                 ix_fname='friktion_ix.csv', 
                 deposit_fname=None, 
                 deposit_cxl_fname=None, 
                 withdraw_fname=None, 
                 withdraw_cxl_fname=None,
                 batch_size_days=14, 
                 batch_size_xfers=50
            ):
        """
        ix_fname:              output csv for instructions
        deposit_fname:         output csv for deposits
        deposit_cxl_fname:     output csv for deposit cancels
        withdraw_fname:        output csv for withdrawals
        withdraw_cxl_fname:    output csv for withdrawal cancels
        batch_size_days:       batch size in days for query to keep query < 10k rows. Use bigger steps for larger data.
        batch_size_transfers:  batch size transactions for query to keep query < 8kb 

        """
        self.volt_program = "VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp"
        self.date_start = date_start
        self.date_end = date_end
        self.ix_fname = ix_fname
        self.deposit_fname = deposit_fname
        self.deposit_cxl_fname = deposit_cxl_fname
        self.withdraw_fname = withdraw_fname
        self.withdraw_cxl_fname = withdraw_cxl_fname
        self.batch_size_days = batch_size_days
        self.batch_size_xfers = batch_size_xfers

    @staticmethod
    def run_query(query):  # A simple function to use requests.post to make the API call.
        headers = {'X-API-KEY': 'BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4'}
        request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
        if request.status_code == 200:
            return request.json()
        else:
            print(request.reason)
            raise Exception('Query failed and return code is {}.{}'.format(request.status_code, query))

    @property
    def ix_query(self):
        return """
            query MyQuery {
              solana {
                instructions(
                  time: {between: ["%s", "%s"]}
                  success: {is: true}
                  programId: {is: "VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp"}
                ) {
                  block {
                    timestamp {
                      iso8601
                    }
                  }
                  log {
                    consumed
                    instruction
                    logs
                  }
                  transaction {
                    signature
                    success
                    feePayer
                  }
                }
              }
            }
        """
        
    @property
    def xfer_query(self):
        return """
            query MyQuery {
              solana(network: solana) {
                transfers(
                  time: {between: ["%s", "%s"]}
                  signature: {in: [%s]}
                ) {
                  instruction {
                    action {
                      name
                      type
                    }
                    callPath
                  }
                  amount(success: {is: true})
                  transaction {
                    signer
                    signature
                  }
                  block {
                    timestamp {
                      iso8601
                    }
                  }
                  currency {
                    name
                    address
                    symbol
                    decimals
                  }
                  sender {
                    address
                    mintAccount
                    type
                  }
                  receiver {
                    address
                    mintAccount
                    type
                  }
                }
              }
            }
        """
        
    def get_ix(self, date_start, date_end):
        """
        Runs graphql instruction query for one date range. 
        """
        query = self.ix_query % (date_start, date_end)
        print(datetime.now(), "retrieving instructions for {} to {}".format(date_start, date_end))
        result = self.run_query(query)
        
        # convert GraphQL json to pandas dataframe
        df = pd.json_normalize(result['data']['solana']['instructions'])
        print(datetime.now(), df.shape[0], "instructions retrieved")
        
        df = df.rename(columns={"block.timestamp.iso8601": "timestamp", 
                           "log.consumed": "computeUnits", 
                           "log.instruction": "instructionType", 
                           "log.logs": "programLogs", 
                           "transaction.signature": "txSignature", 
                           "transaction.success": "txSuccess", 
                           "transaction.feePayer": "txSigner"})
        return df
    
    def get_ix_batch(self):
        """
        Batch the instruction retrieval. Save the shit Drop duplicates. 

        """
        # Create output file if doesn't exist
        if self.ix_fname and exists(self.ix_fname):
            df_ix = pd.read_csv(self.ix_fname)
        else:
            df_ix = pd.DataFrame()
        
        # Batch the days up nice and good so the graphql API calls don't bitch
        dates_batched = pd.date_range(self.date_start, self.date_end, freq='7D')
        dates_batched = [str(x.isoformat()) for x in dates_batched.append(pd.DatetimeIndex([self.date_end]))]
        date_ranges = list(zip(dates_batched, dates_batched[1:]))
        
        for date_range in date_ranges:
            assert len(date_range)==2
            data = self.get_ix(date_range[0], date_range[1])
            df_ix = df_ix.append(data, ignore_index=True)
        
        df_ix.drop_duplicates().to_csv(self.ix_fname, index=False)
        print(datetime.now(), "final data size: ", df_ix.shape[0])
        self.df_ix = df_ix
    
    def parse_deposits(self):
        instructionType = 'Deposit'
        
        #        # Create output file if doesn't exist
        if self.ix_fname and exists(self.ix_fname):
            df_ix = pd.read_csv(self.ix_fname)
        else:
            df_ix = pd.DataFrame()
        
        if temp.empty:
            print(datetime.now(), "instructionType was not found in the data... breaking")
            return
    
        
        
    def parse_withdrawal(self):
        pass
    
    def parse_deposit_cancel(self):
        pass
    
    def parse_withdrawal_cancel(self):
        pass
    
    @staticmethod
    def get_batch(iterable, n=1):
        """
        Takes in an iterable and returns an iterable of iterables with len==x
        """
        idxs = []
        l = len(iterable)
        for idx in range(0, l, n):
            print(idx)
            idxs.append(iterable[idx:min(idx+n, l)])
        return idxs
        
    def format_txs_for_query(self, tx_signatures):
        """
        Batches a list of transactions into a list of string formatted transactions for querying. 
        Each of these strings contain (n=self.batch_size_xfers) unique transaction IDs.
        """
        batched_signatures = self.get_batch(tx_signatures, self.batch_size_xfers)
        
        def format_txs(x):
            return str(x)[1:-1].replace("\'", "\"").replace("\n", "")
        
        tx_str = list(map(format_txs, batched_signatures))
        return tx_str
        
    def get_batched_xfers(self, instructionType):
        """
        Get all transfers corresponding to a specific instructionType from Graphql query. 
        """
        temp = self.df_ix.query("instructionType=='%s'" % instructionType)
        
        tx_signatures = list(temp["txSignature"].unique())
        print(len(tx_signatures))
        tx_strs = self.format_txs_for_query(tx_signatures)
        
        df_xfer = pd.DataFrame()
        
        for tx_str in tx_strs:
            query = self.xfer_query % (self.date_start, self.date_end, tx_str)
            result = self.run_query(query)
            df = pd.json_normalize(result['data']['solana']['transfers'])
            print(datetime.now(), df.shape[0], "transfers retrieved")
            df_xfer = df_xfer.append(df)
            
        print(df_xfer)
        return df_xfer

    #         df = df.rename(columns={"block.timestamp.iso8601": "timestamp", 
    #                            "log.consumed": "computeUnits", 
    #                            "log.instruction": "instructionType", 
    #                            "log.logs": "programLogs", 
    #                            "transaction.signature": "txSignature", 
    #                            "transaction.success": "txSuccess", 
    #                            "transaction.feePayer": "txSigner"})

In [427]:
date_start = "2022-03-01T00:00:00Z"
date_end = "2022-03-06T00:00:00Z"

In [428]:
x = MyPortfolio(date_start, date_end)

In [429]:
x.get_ix_batch()

2022-03-29 23:45:00.015338 retrieving instructions for 2022-03-01T00:00:00+00:00 to 2022-03-06T00:00:00+00:00
2022-03-29 23:45:07.058948 2836 instructions retrieved
2022-03-29 23:45:07.128711 final data size:  8884


In [430]:
x.get_batched_xfers("Deposit")

219
0
50
100
150
200
2022-03-29 23:45:08.819413 44 transfers retrieved
2022-03-29 23:45:10.393282 0 transfers retrieved
2022-03-29 23:45:11.882972 0 transfers retrieved
2022-03-29 23:45:13.466998 0 transfers retrieved
2022-03-29 23:45:14.550486 0 transfers retrieved
         amount instruction.action.name instruction.action.type  \
0      0.002039            closeAccount            closeAccount   
1      0.002039            closeAccount            closeAccount   
2      0.002039            closeAccount            closeAccount   
3      0.002039            closeAccount            closeAccount   
4      0.002039            closeAccount            closeAccount   
5      0.002039            closeAccount            closeAccount   
6      0.002039            closeAccount            closeAccount   
7      0.002039            closeAccount            closeAccount   
8      1.492039           createAccount           createAccount   
9      2.002039           createAccount           createAccount

,amount,instruction.action.name,instruction.action.type,instruction.callPath,transaction.signer,transaction.signature,block.timestamp.iso8601,currency.name,currency.address,currency.symbol,currency.decimals,sender.address,sender.mintAccount,sender.type,receiver.address,receiver.mintAccount,receiver.type
0,0.002039,closeAccount,closeAccount,4,4PTiusSXr57BDZjfFCCyeyMKqEbmGHc4ZAprgXkAwQPs,2mqkZNkgPLdb19Wrc64teCTg5DtYskEwEDdbhGNEJhgqtjwxv2933CXRAxLcTpgfGmE6ZP9WmoGoxJ3KdPqeNh6t,2022-03-05T21:43:23Z,Solana,-,SOL,9,CSZrVWi9VDp3CVV5uKDTtpSFqJmop8mFBrk2F4NsEtWu,CSZrVWi9VDp3CVV5uKDTtpSFqJmop8mFBrk2F4NsEtWu,account,4PTiusSXr57BDZjfFCCyeyMKqEbmGHc4ZAprgXkAwQPs,4PTiusSXr57BDZjfFCCyeyMKqEbmGHc4ZAprgXkAwQPs,account
1,0.002039,closeAccount,closeAccount,4,4PTiusSXr57BDZjfFCCyeyMKqEbmGHc4ZAprgXkAwQPs,2mqkZNkgPLdb19Wrc64teCTg5DtYskEwEDdbhGNEJhgqtjwxv2933CXRAxLcTpgfGmE6ZP9WmoGoxJ3KdPqeNh6t,2022-03-05T21:43:23Z,Wrapped SOL,So11111111111111111111111111111111111111112,SOL,9,4PTiusSXr57BDZjfFCCyeyMKqEbmGHc4ZAprgXkAwQPs,CSZrVWi9VDp3CVV5uKDTtpSFqJmop8mFBrk2F4NsEtWu,account,,,
2,0.002039,closeAccount,closeAccount,4,9LJ6KUmwEgRqqxJRhBfsdTgdprpMfAaRzJAwYkksaErn,3MRnEDL4Fi3vcNP9UJgBfDkq2XBRzWFy6bJmxR5P3D4N1bP7GoGvZrTuLsseDrqQxjtddcoWU6gcPLWFaaznsYS4,2022-03-05T23:25:34Z,Solana,-,SOL,9,92aWoeSKmCf25pTBanSGWpCieZETodPWfDf8mDo92Mf6,92aWoeSKmCf25pTBanSGWpCieZETodPWfDf8mDo92Mf6,account,9LJ6KUmwEgRqqxJRhBfsdTgdprpMfAaRzJAwYkksaErn,9LJ6KUmwEgRqqxJRhBfsdTgdprpMfAaRzJAwYkksaErn,account
3,0.002039,closeAccount,closeAccount,4,9LJ6KUmwEgRqqxJRhBfsdTgdprpMfAaRzJAwYkksaErn,3MRnEDL4Fi3vcNP9UJgBfDkq2XBRzWFy6bJmxR5P3D4N1bP7GoGvZrTuLsseDrqQxjtddcoWU6gcPLWFaaznsYS4,2022-03-05T23:25:34Z,Wrapped SOL,So11111111111111111111111111111111111111112,SOL,9,9LJ6KUmwEgRqqxJRhBfsdTgdprpMfAaRzJAwYkksaErn,92aWoeSKmCf25pTBanSGWpCieZETodPWfDf8mDo92Mf6,account,,,
4,0.002039,closeAccount,closeAccount,4,AMFkHX3aGXWMCEjF6mrhxJrHB4xuBSMh7cjZJEc28ZTi,3pqAUZRC9x4cakxhXE4BcNUzx9CaNzYnxmSAruz79jy6NodxpP6Riq9waS982PEGPnhRPP8f4afmwxf4S6PG19oG,2022-03-05T21:43:11Z,Solana,-,SOL,9,7BnHztWkU973nNQgiNRT6N18jLxZA3UZt6oTHgRSwZJT,7BnHztWkU973nNQgiNRT6N18jLxZA3UZt6oTHgRSwZJT,account,AMFkHX3aGXWMCEjF6mrhxJrHB4xuBSMh7cjZJEc28ZTi,AMFkHX3aGXWMCEjF6mrhxJrHB4xuBSMh7cjZJEc28ZTi,account
5,0.002039,closeAccount,closeAccount,4,AMFkHX3aGXWMCEjF6mrhxJrHB4xuBSMh7cjZJEc28ZTi,3pqAUZRC9x4cakxhXE4BcNUzx9CaNzYnxmSAruz79jy6NodxpP6Riq9waS982PEGPnhRPP8f4afmwxf4S6PG19oG,2022-03-05T21:43:11Z,Wrapped SOL,So11111111111111111111111111111111111111112,SOL,9,AMFkHX3aGXWMCEjF6mrhxJrHB4xuBSMh7cjZJEc28ZTi,7BnHztWkU973nNQgiNRT6N18jLxZA3UZt6oTHgRSwZJT,account,,,
6,0.002039,closeAccount,closeAccount,4,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,5FLoFGi8u7sL9Bo6mwnzvLJzorqeLEdqDcfkXYDYyHTMSUDF3SXjZTGWMreopv29vX4eCKB4wevPBv8FBet7wdgk,2022-03-05T21:22:18Z,Solana,-,SOL,9,3VmzZ9uveuDogQKVLoCGpci3mU1mmaY5T8ko486KsaPx,3VmzZ9uveuDogQKVLoCGpci3mU1mmaY5T8ko486KsaPx,account,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,account
7,0.002039,closeAccount,closeAccount,4,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,5FLoFGi8u7sL9Bo6mwnzvLJzorqeLEdqDcfkXYDYyHTMSUDF3SXjZTGWMreopv29vX4eCKB4wevPBv8FBet7wdgk,2022-03-05T21:22:18Z,Wrapped SOL,So11111111111111111111111111111111111111112,SOL,9,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE,3VmzZ9uveuDogQKVLoCGpci3mU1mmaY5T8ko486KsaPx,account,,,
8,1.492039,createAccount,createAccount,0,4PTiusSXr57BDZjfFCCyeyMKqEbmGHc4ZAprgXkAwQPs,2mqkZNkgPLdb19Wrc64teCTg5DtYskEwEDdbhGNEJhgqtjwxv2933CXRAxLcTpgfGmE6ZP9WmoGoxJ3KdPqeNh6t,2022-03-05T21:43:23Z,Solana,-,SOL,9,4PTiusSXr57BDZjfFCCyeyMKqEbmGHc4ZAprgXkAwQPs,4PTiusSXr57BDZjfFCCyeyMKqEbmGHc4ZAprgXkAwQPs,account,CSZrVWi9VDp3CVV5uKDTtpSFqJmop8mFBrk2F4NsEtWu,CSZrVWi9VDp3CVV5uKDTtpSFqJmop8mFBrk2F4NsEtWu,account
9,2.002039,createAccount,createAccount,0,9LJ6KUmwEgRqqxJRhBfsdTgdprpMfAaRzJAwYkksaErn,3MRnEDL4Fi3vcNP9UJgBfDkq2XBRzWFy6bJmxR5P3D4N1bP7GoGvZrTuLsseDrqQxjtddcoWU6gcPLWFaaznsYS4,2022-03-05T23:25:34Z,Solana,-,SOL,9,9LJ6KUmwEgRqqxJRhBfsdTgdprpMf

In [431]:
zz = pd.read_csv("friktion_ix.csv")
zz.loc[zz.instructionType=="Deposit"]

,timestamp,computeUnits,instructionType,programLogs,txSignature,txSuccess,txSigner
803,2022-03-03T08:20:48Z,140069,Deposit,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 140069 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,2SbCu8mQxJ6g4Zk1eePHpgom4v3xXHNkswZh5nv1dsmh48qwuR6UzM7ngKqttwZaHFbdhYqa5HvXwg283BgzS5r4,True,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE
2815,2022-03-05T21:22:18Z,87308,Deposit,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program log: validating Deposit context;Program log: beginning deposit...;Program log: user underlying = 100000, deposit amount = 100000, vault capacity = 290000000000000;Program log: post deposit estimated underlying: 198984912191755, vault capacity: 290000000000000;Program log: pending deposit;Program log: transferring 100000 tokens from 3VmzZ9uveuDogQKVLoCGpci3mU1mmaY5T8ko486KsaPx to 5GHuBr9njP3KBqXYS7KWRX6z97eYCmqBpn7b5Ae4jeWj;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 87308 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",5FLoFGi8u7sL9Bo6mwnzvLJzorqeLEdqDcfkXYDYyHTMSUDF3SXjZTGWMreopv29vX4eCKB4wevPBv8FBet7wdgk,True,DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE
2818,2022-03-05T21:43:11Z,93263,Deposit,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 93263 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,3pqAUZRC9x4cakxhXE4BcNUzx9CaNzYnxmSAruz79jy6NodxpP6Riq9waS982PEGPnhRPP8f4afmwxf4S6PG19oG,True,AMFkHX3aGXWMCEjF6mrhxJrHB4xuBSMh7cjZJEc28ZTi
2819,2022-03-05T21:43:23Z,100709,Deposit,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 100709 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,2mqkZNkgPLdb19Wrc64teCTg5DtYskEwEDdbhGNEJhgqtjwxv2933CXRAxLcTpgfGmE6ZP9WmoGoxJ3KdPqeNh6t,True,4PTiusSXr57BDZjfFCCyeyMKqEbmGHc4ZAprgXkAwQPs
2820,2022-03-05T21:44:44Z,105003,Deposit,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program log: validating Deposit context;Program log: beginning deposit...;Program log: user underlying = 3816, deposit amount = 10, vault capacity = 1800000000;Program log: post deposit estimated underlying: 966236063, vault capacity: 1800000000;Program log: pending deposit;Program log: transferring 10 tokens from 2MtkHFTQE167G9R76KHD37fAe44wGwnopU2CLNR3ipPx to 6SyyDyCLoCiCnwjWwCVb7sputaZrFhZkfEvw8sRS1QdD;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 105003 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",5VsyoDn63e6ZyjxAVriTXv7dY5BeuiPyr6wr3y5qpCsy74ZRbCK4hhk8FUfMh4i17QFk4HxKZA3rnkGui6bjg3Y3,True,beer2DnvzpGjPWY5uFR729QmovL6U727mUTkLZjGtpV
...,...,...,...,...,...,...,...
6037,2022-03-05T22:29:18Z,92029,Deposit,"Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Deposit;Program log: validating Deposit context;Program log: beginning deposit...;Program log: user underlying = 999943862, deposit amount = 999943862, vault capacity = 150000000000;Program log: post deposit estimated underlying: 119851360013, vault capacity: 150000000000;Program log: pending deposit;Program log: transferring 999943862 tokens from CGJngL5FEb66v682VuMv6HvHCByAhApPgMya5JetyUwD to GVViqUdyWWjKRM29VG7rwBshkcaRcoiurAJp8TZCCiaz;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 92029 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success",61eBSqcd72ufDwVCPpXprsuqeYe4dZoGdtzMPZXwkhyet5pLVcPPsxjZRwJrtG6cTGBrGBbbqY2wFhpT6nFmrWJo,True,FAmwbKLQqNEqAmaRio6bHuKNGx1zp7t18FfgEBNrpoYE
6042,2022-03-05T22:50:56Z,99017,De